**Goal: Variational Inference on parameters of Dirichlet distribution**

Model:
$$
\begin{aligned}
y_i &\sim \text{ProjGamma}_p(Y\mid\alpha)\\
\log\alpha &\sim \text{MVNormal}(\log\alpha\mid \log(0.5), I_d)
\end{aligned}
$$

In [1]:
import silence_tensorflow.auto
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import bijectors as tfb
from numpy.random import gamma

from tfprojgamma import ProjectedGamma
# Set random seeds for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [2]:
# Set up shape parameters
alpha_true = gamma(size = 5, shape = 1.5)
print(alpha_true)
dist1 = ProjectedGamma(alpha_true, 10)
Yp    = tf.cast(dist1.sample(200), tf.float32)

[3.94761907 0.62279234 0.68411215 0.34912511 4.2482095 ]


In [3]:
# prior shape parameters
log_alpha_0 = tf.ones(5, dtype = tf.float32) * np.log(0.5)

**Define the Joint Distribution**

In [4]:
# define generator
def generative_model(log_alpha_0, n_samples):
    log_alpha = yield tfd.JointDistributionCoroutine.Root(
        tfd.MultivariateNormalDiag(
            loc = log_alpha_0, scale_diag = tf.ones(5, dtype = tf.float32), name = 'log_alpha'
            ),
        )
    Yp = yield tfd.Sample(
        ProjectedGamma(concentration = tf.exp(log_alpha) * tf.ones((n_samples, 5), dtype = tf.float32)),
        name = 'Yp',
        )

model_joint = tfd.JointDistributionCoroutineAutoBatched(
    lambda: generative_model(log_alpha_0, 200),
    )

model_joint_log_prob = lambda log_alpha: model_joint.log_prob(log_alpha, Yp)

**Verifying the structure of the joint model**

In [5]:
model_joint

<tfp.distributions.JointDistributionCoroutineAutoBatched 'JointDistributionCoroutineAutoBatched' batch_shape=[] event_shape=StructTuple(
  log_alpha=[5],
  Yp=[200, 5]
) dtype=StructTuple(
  log_alpha=float32,
  Yp=float32
)>

**Mean Field Variational Bayes -- Independence between columns**
$$
\log\alpha \sim \prod_{\ell = 1}^d\text{Normal}(\log\alpha_{\ell} \mid \mu_{q\ell}, \sigma_{q\ell})
$$

In [6]:
q_mu = tf.Variable(log_alpha_0, dtype = tf.float32)
q_scale = tfp.util.TransformedVariable(np.ones(5), tfb.Exp(), dtype = tf.float32)

surrogate_posterior = tfd.MultivariateNormalDiag(loc = q_mu, scale_diag = q_scale, name = 'surrogate 1')

with tf.GradientTape() as g:
    samples = surrogate_posterior.sample(100)
    neg_elbo = -tf.reduce_mean(model_joint_log_prob(samples) - surrogate_posterior.log_prob(samples))
print(g.gradient(neg_elbo, surrogate_posterior.trainable_variables)) # exists!

(<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-163.42444,  393.58337,  296.38065,  527.06805, -194.55045],
      dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 84.23479 , 691.3565  , 537.5379  , 729.03815 ,  68.810196],
      dtype=float32)>)


In [7]:
path = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn = model_joint_log_prob,
    surrogate_posterior = surrogate_posterior,
    optimizer = tf.optimizers.Adam(.2),
    num_steps = 1000,
    sample_size = 500,
    )

print(tf.exp(q_mu)) # This appears to have worked; the values end in *rougly* the right place.
print(q_scale**2)

tf.Tensor([3.9614034 0.5935092 0.6732101 0.3893673 4.068376 ], shape=(5,), dtype=float32)
tf.Tensor([0.00178767 0.00397885 0.00379829 0.00435149 0.00177991], shape=(5,), dtype=float32)


**Gaussian Variational Bayes -- Dependence Between Columns**
$$
\log\alpha \sim \text{MVNormal}(\log\alpha \mid \mu_q, \Sigma_q)
$$

In [8]:
# New Style: Make the variational Parameters
q_nu = tf.Variable(tf.zeros(5, dtype = tf.float32), name = 'Mu Surrogate (mean of log alpha)')
cholbijector = tfb.FillScaleTriL(diag_bijector = tfb.Exp())
q_Lu = tfp.util.TransformedVariable(tf.eye(5), bijector = cholbijector)

surrogate_posterior_mvnorm = tfd.MultivariateNormalTriL(loc = q_nu, scale_tril = q_Lu)

with tf.GradientTape() as g:
    samples = surrogate_posterior_mvnorm.sample(100)
    neg_elbo = -tf.reduce_mean(model_joint_log_prob(samples) - surrogate_posterior_mvnorm.log_prob(samples))
print(g.gradient(neg_elbo, surrogate_posterior_mvnorm.trainable_variables)) # Exists!

(<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-167.1391 ,  843.6687 ,  565.7811 , 1421.4977 , -153.87805],
      dtype=float32)>, <tf.Tensor: shape=(15,), dtype=float32, numpy=
array([ 338.59695 , -125.0202  ,   26.117214,  -40.666622, -135.07663 ,
        171.36061 , 2645.6987  , -998.0686  , -920.1034  ,  452.4206  ,
        167.13881 , 1390.3918  , 1060.2035  ,   68.15751 ,  274.95932 ],
      dtype=float32)>)


In [9]:
path_mvnorm = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn = model_joint_log_prob,
    surrogate_posterior = surrogate_posterior_mvnorm,
    optimizer = tf.optimizers.Adam(.2),
    num_steps = 1000,
    sample_size = 500,
    )
print(tf.exp(q_nu)) # This gives the same basic response as previous.

tf.Tensor([3.8994887  0.5955216  0.6662088  0.38724408 4.1442165 ], shape=(5,), dtype=float32)


In [17]:
(q_Lu.numpy() @ q_Lu.numpy().T > 0)

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

In [24]:
(q_Lu.numpy() @ q_Lu.numpy().T)

array([[0.00384308, 0.00172062, 0.00245882, 0.00183036, 0.00274806],
       [0.00172062, 0.00478812, 0.001474  , 0.00106701, 0.00123463],
       [0.00245882, 0.001474  , 0.00548552, 0.00076655, 0.00180139],
       [0.00183036, 0.00106701, 0.00076655, 0.00534088, 0.00147093],
       [0.00274806, 0.00123463, 0.00180139, 0.00147093, 0.00374892]],
      dtype=float32)

I guess it makes some sense that the posterior covariance between parameters of the Dirichlet would be positive, despite the covariance between *values* of the Dirichlet being negative.